## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import OpenWeather_API_Key
from config import GoogleMaps_API_Key

# Configure gmaps API key
gmaps.configure(api_key=GoogleMaps_API_Key)

In [4]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp_input = float((input(f'What is you desired Min Temperature?')))
max_temp_input = float((input(f'What is you desired Max Temperature?')))


What is you desired Min Temperature?65
What is you desired Max Temperature?76


In [5]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
input_file = "../Weather_Database/WeatherPY_Database.csv"
citywx_df = pd.read_csv(input_file)

# Determine if there are any empty rows.
total_rows = citywx_df.count()

# country column has 707 entries. all others have 711
citywx_df.dropna(inplace=True)
citywx_df_loc = citywx_df.loc[(citywx_df["Max_Temp_(F)"] >= min_temp_input) & (citywx_df["Max_Temp_(F)"] <= max_temp_input)]
citywx_df_loc.head(20)

,City_ID,City,Country,Date,Lat,Lon,Wx_Desc,Max_Temp_(F),Humidity_%,Cloud_%,Wind_Spd
1,1,Hermanus,ZA,2022-03-20 23:29:45,-34.4187,19.2345,clear sky,68.81,87,0,9.51
2,2,Carnarvon,AU,2022-03-20 23:29:46,-24.8667,113.6333,few clouds,73.47,83,14,21.85
3,3,Pringsewu,ID,2022-03-20 23:29:46,-5.3582,104.9744,overcast clouds,73.15,95,92,3.31
6,6,Laguna,US,2022-03-20 23:29:48,38.4210,-121.4238,clear sky,73.69,17,0,19.57
14,14,Port Alfred,ZA,2022-03-20 23:29:52,-33.5906,26.8910,overcast clouds,71.20,80,94,8.57
17,17,Hirara,JP,2022-03-20 23:29:54,24.8000,125.2833,broken clouds,71.46,88,75,5.75
21,21,Salalah,OM,2022-03-20 23:29:56,17.0151,54.0924,clear sky,69.89,32,0,3.44
25,25,Kutum,SD,2022-03-20 23:29:57,14.2000,24.6667,broken clouds,67.82,6,75,7.87
37,37,Namibe,AO,2022-03-20 23:30:03,-15.1961,12.1522,scattered clouds,73.33,79,36,5.10
40,40,Ahipara,NZ,2022-03-20 23:30:04,-35.1667,173.1667,scattered clouds,73.56,66,26,5.95


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = citywx_df_loc[["City", "Country", "Max_Temp_(F)", "Wx_Desc", "Lat", "Lon"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel_Name"] = ""
hotel_df.reset_index(inplace=True)
hotel_df.pop('index')
hotel_df.head(10)


,City,Country,Max_Temp_(F),Wx_Desc,Lat,Lon,Hotel_Name
0,Axim,GH,79.74,overcast clouds,4.8699,-2.2405,
1,Hermanus,ZA,68.81,clear sky,-34.4187,19.2345,
2,Carnarvon,AU,73.47,few clouds,-24.8667,113.6333,
3,Pringsewu,ID,73.15,overcast clouds,-5.3582,104.9744,
4,Namatanai,PG,84.43,overcast clouds,-3.6667,152.4333,
5,Laguna,US,73.69,clear sky,38.4210,-121.4238,
6,Arraial Do Cabo,BR,76.55,broken clouds,-22.9661,-42.0278,
7,Port Alfred,ZA,71.20,overcast clouds,-33.5906,26.8910,
8,Coolum Beach,AU,79.34,clear sky,-26.5333,153.1000,
9,Hirara,JP,71.46,broken clouds,24.8000,125.2833,


In [6]:
# Iterate through the hotel DataFrame.
for i in range(0, len(hotel_df)):
    # 6c. Get latitude and longitude from DataFrame
    location = str(hotel_df["Lat"][i]) + ", " + str(hotel_df["Lon"][i])
    params = {
    "location": location,
    "radius": 5000,
    "type": "lodging",
    "key": GoogleMaps_API_Key}
    
# Build URL using the Google Maps API
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[i, "Hotel_Name"] = response["results"][0]["name"]
    except:
        print(f'Hotel Not Found... Skipping')
        pass
print(f'Completed Hotel Search')

Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Completed Hotel Search


In [7]:
# Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace=True)
hotel_df = hotel_df[hotel_df["Hotel_Name"] != ""]



In [16]:
# Create the output File (CSV)
output_data_file = "Vacation_Search/Hotel_Search_Results.csv"

# Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [9]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel:</dt><dd>{Hotel_Name}</dd>
<dt>City:</dt><dd>{City}</dd>
<dt>Country:</dt><dd>{Country}</dd>
<dt>Temp:</dt><dd>{Max_Temp_(F)} °F</dd>
<dt>Wx Description:</dt><dd>{Wx_Desc}</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[["Lat", "Lon"]]

In [15]:
# Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lon"]]
temp = hotel_df["Max_Temp_(F)"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=temp, dissipating=False, max_intensity=300, point_radius=6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))